### Réalisé par : SKOURI Youssef.

Dans ce notebook, nous travaillons sur The Movies Dataset de Kaggle, un ensemble de données contenant diverses informations sur les films, dont les overviews.

L’objectif est d’appliquer des techniques de traitement automatique du langage naturel (NLP) afin d’étudier les relations entre les mots présents dans les descriptions des films.

Pour cela, nous utiliserons la librairie Gensim, en particulier le modèle Word2Vec, qui permet d’apprendre des représentations vectorielles de mots et d’explorer leurs similarités et analogies.

## Importations utiles

In [36]:
import pandas as pd
import gensim
from gensim.models import Word2Vec
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

In [37]:
films = pd.read_csv("/Users/mac/Desktop/THEMOVIES/movies_metadata.csv")

/var/folders/l4/q_qcck5x0mx0d5ly8ycdhcwh0000gn/T/ipykernel_2796/601302881.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  films = pd.read_csv("/Users/mac/Desktop/THEMOVIES/movies_metadata.csv")


In [38]:
films.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [39]:
len(films)

45466

In [40]:
films.overview.isna().sum()

954

## Les valeurs manquantes de la variable Overview ne concernent que 954 (2%) donc on va procéder par une suppression.

### Supprimer valeurs manquantes / vides

In [41]:
films = films.dropna(subset=['overview'])
films = films[films['overview'].str.strip() != ""]

In [42]:
films.overview.isna().sum()

0

### Mettre les texts en minuscule 

In [43]:
films['overview'] = films['overview'].str.lower()

### Retirer ponctuation

In [44]:
films['overview'] = films['overview'].str.replace(r'[^\w\s]', ' ', regex=True)

### Tokenisation

#### On télécharge le modèle pré-entraîné PUNKT, puison applique word_tokenize pour transformer chaque overview en liste de mots.

In [45]:
nltk.download('punkt')
films['tokens'] = films['overview'].apply(nltk.word_tokenize)

[nltk_data] Downloading package punkt to /Users/mac/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


### Retrait stopwords

#### Suppression des mots très fréquentsqui n’apportent presque aucune information sémantique pour réduire le bruit et d’améliorer la qualité du corpus.

In [46]:
nltk.download('stopwords')
stops = set(stopwords.words('english'))
films['tokens'] = films['tokens'].apply(lambda x: [w for w in x if w not in stops])

[nltk_data] Downloading package stopwords to /Users/mac/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### Lemmatisation

#### On utilise la lemmatisation plutôt que le stemming car elle renvoie des mots réels et préserve mieux le sens linguistique

In [47]:
nltk.download('wordnet')
lem = WordNetLemmatizer()
films['tokens'] = films['tokens'].apply(lambda x: [lem.lemmatize(w) for w in x])

[nltk_data] Downloading package wordnet to /Users/mac/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


## Vérification rapide du résultat 

In [48]:
films['tokens'].head()

0    [led, woody, andy, toy, live, happily, room, a...
1    [sibling, judy, peter, discover, enchanted, bo...
2    [family, wedding, reignites, ancient, feud, ne...
3    [cheated, mistreated, stepped, woman, holding,...
4    [george, bank, recovered, daughter, wedding, r...
Name: tokens, dtype: object

### Construire le corpus

In [49]:
corpus = films['tokens'].tolist()

### Entraîner Word2Vec

À cette étape, on entraine le modèle sur le corpus crée précédemment.

In [50]:
model = Word2Vec(
    sentences=corpus,
    vector_size=100,
    window=5,
    min_count=2,
    workers=4,
    epochs=30
)

#### sentences : le corpus tokenisé (liste de listes de mots)
#### vector_size : dimension des vecteurs de mots (100 = standard)
#### window : taille de la fenêtre de contexte autour d’un mot
#### min_count : ignore les mots apparaissant moins de 2 fois
#### workers : nombre de cœurs CPU utilisés pour l’entraînement
#### epochs : nombre de passages sur le corpus, on a choisi 30 pour assurer le bon apprentissage. Bien évidemment, cela peut causer des problèmes d'over-fitting. Mais, j'estime que c'est hors le cadre de ce TP.

## Similarité et Analogie 

On teste ici le mot amour (love) pour vérifier si le modèle renvoie des similarités logiques.

In [51]:
model.wv.most_similar("love")

[('inlove', 0.5405173301696777),
 ('passion', 0.5083723068237305),
 ('apart', 0.5063254237174988),
 ('asleep', 0.4936375916004181),
 ('feeling', 0.48407596349716187),
 ('sorrow', 0.46239885687828064),
 ('romance', 0.454603374004364),
 ('affection', 0.45336613059043884),
 ('spell', 0.44745293259620667),
 ('charm', 0.4436807334423065)]

Les résultats semblent logiques !

On teste un exemple assez classique : woman+king-man pour vérifier si le modèle arrive à prédire correctement

In [52]:
model.wv.most_similar(positive=["woman", "king"], negative=["man"])

[('queen', 0.5455237627029419),
 ('princess', 0.530830979347229),
 ('musketeer', 0.4701135754585266),
 ('ruler', 0.44445091485977173),
 ('crowned', 0.43960386514663696),
 ('salome', 0.43732866644859314),
 ('artagnan', 0.4261471927165985),
 ('prince', 0.4183848798274994),
 ('gwang', 0.4178825616836548),
 ('jester', 0.4145362973213196)]

Queen/Princess sont évidemment les mots attendus !